In [1]:
pip install sastrawi

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd 
import string 
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory,StopWordRemover

import re

In [16]:
dataset_sg = pd.read_csv('dum.csv')

In [17]:
dataset_sg

,ulasan,label,pelayanan,fasilitas,wahana,kuliner
0,"jalannya hancur<br> tidak bisa dilewati, tukan...",0,1,1,0,0
1,wahananya bagus keretanya juga menyenangkan,1,0,0,1,0
2,"makanan dan minumannya mahal dan tidak enak, d...",0,0,1,0,1
3,"adminnya tidak ramah, tukang parkirnya juga ti...",0,1,0,0,0
4,bagus juga untuk dibuat liburan untuk keluarga,1,0,0,0,0
5,"kolam sangat kotor, dan cafenya kurang banyak ...",0,0,0,1,1
6,"atvnya rusak, toiletnya bau dan kotor",0,0,1,1,0
7,makanannya enak dan tempatnya menyenangkan unt...,1,0,0,0,1
8,kurang tempat berteduh dan cuacanya panas sekali,0,0,1,0,0
9,toiletnya bersih dan kolamnya juga bagus,1,0,1,1,0


In [18]:
#Case Folding
data_baru = dataset_sg['ulasan'].str.lower()
dataset_sg['ulasan'] = data_baru

In [19]:
#Menghapus tag <br>
dataset_sg['ulasan'] = dataset_sg['ulasan'].replace("<br>", " ")

In [20]:
#Hapus angka dan simbol
def hapus_angka_simbol(data):
    data_bersih = re.sub('[^a-zA-Z]', ' ', data)
    return data_bersih

dataset_sg['ulasan'] = dataset_sg['ulasan'].apply(hapus_angka_simbol)

In [21]:
#baca slangword
slang_dict = pd.read_excel('sw.xlsm')

# Buat dictionary untuk mapping slangword dan standar
slang_map = dict(zip(slang_dict['slang'], slang_dict['formal']))

In [22]:
# Membuat fungsi untuk mencari dan mengganti slangword
def hapus_slangword(text):
    for key in slang_map:
        text = re.sub(rf"\b{key}\b", slang_map[key], text)
    return text

# Mengganti slangword di kolom tertentu
dataset_sg['ulasan'] = dataset_sg['ulasan'].apply(hapus_slangword)

In [23]:
#Tokenizing with unigram
def token_unigram(data):
    datas = word_tokenize(data)
    return [token for token in datas if len(token) > 1]

dataset_sg['ulasan'] = dataset_sg['ulasan'].apply(token_unigram)

In [24]:
#Remove Indonesian Stopword
factory = StopWordRemoverFactory() 
stop = set(factory.get_stop_words())
stop.discard('tidak')

In [25]:
def hapus_stop(text): 
    txt = [token for token in text if token not in stop]
    txt_b = ' '.join(txt)
    return txt_b

dataset_sg['ulasan'] = dataset_sg['ulasan'].apply(hapus_stop)

In [26]:
#Stemming with Sastrawi
f = StemmerFactory()
stem = f.create_stemmer()

def stemming(data):
    data = stem.stem(data)
    return data

dataset_sg['ulasan'] = dataset_sg['ulasan'].apply(stemming)

In [27]:
dataset_sg['ulasan']

0    jalan hancur baru tidak lewat tukang parkir ti...
1                           wahana bagus kereta senang
2    makan minum mahal tidak enak jalan hancur tida...
3       adminnya tidak ramah tukang parkir tidak sopan
4                            bagus buat libur keluarga
5    kolam sangat kotor cafenya kurang banyak pilih...
6                        atvnya rusak toilet bau kotor
7                     makan enak tempat senang kunjung
8               kurang tempat teduh cuaca panas sekali
9                            toilet bersih kolam bagus
Name: ulasan, dtype: object

In [28]:
dataset_sg.to_csv('dum_ap.csv', index=False)